In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
results_df = pd.read_csv("data/formula-1/results.csv")

In [3]:
results_df

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25135,25141,1060,849,3,6,18,16,16,16,0.0,70,\N,\N,62,11,1:08.520,226.865,11
25136,25142,1060,20,117,5,11,17,17,17,0.0,69,\N,\N,51,8,1:08.420,227.196,4
25137,25143,1060,854,210,47,19,18,18,18,0.0,69,\N,\N,56,18,1:09.394,224.007,12
25138,25144,1060,853,210,9,20,19,19,19,0.0,69,\N,\N,49,19,1:09.757,222.842,12


In [ ]:
# Pilotos com mais pontos ao longo do tempo
# precisarei contabilizar as vitórias e aplicar a pontuação atual para ficar justo